In [21]:
import random as rnd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
import time
import pandas as pd

In [6]:
def val(P,x):
    return P[0]+P[1]*x+P[2]*x*x
def grad(P,x):
    return P[1]+P[2]*2*x

In [ ]:
st=time.time()
N,A=50,0
breadth=4
ITER=800
T=0.01
fout=open('ret/simple_mean(N,A,breadth,ITER,T={},{},{},{},{}).txt'.format(N,A,breadth,ITER,T),'w')
#fout.write('N,A,breadth,ITER,T={},{},{},{},{}\n'.format(N,A,breadth,ITER,T))
fout.write('seed,opt,mean,median\n')
for seed in range(1,101):
    rnd.seed(seed)

    graph=[[j for j in range(max(0,i-breadth),min(i+breadth+1,N))] for i in range(N)]
    for a in range(A):
        graph.append([i for i in range(N)])
        for i in range(N):
            graph[i].append(len(graph)-1)
    Ps=[]
    for i in range(N):
        a=rnd.uniform(0,1)
        v=rnd.uniform(-100,100)
        #a(x-v)^2=ax^2-2avx+av^2
        Ps.append([a*v*v+rnd.uniform(-1000,1000),-2*a*v,a])

    locsversions=[]
    locs=[rnd.uniform(-400,400) for i in range(N)]
    for a in range(A):
        locs.append(10000)
    for rep in range(ITER):
        locsversions.append(locs.copy())
        nlocs=[locs[i]-T*grad(Ps[i],locs[i]) for i in range(N)]
        for a in range(A):
            nlocs.append(10000) #basic adversary: always send this big number to all neighbors
        for i in range(N):
            #TODO: use smarter use of neighbors' values?
            tmp=[nlocs[j] for j in graph[i]]
            tmp.sort()
            locs[i]=np.mean(tmp)#[1:-1])
    locsversions.append(locs)

    tP=[0 for i in range(3)]
    for P in Ps:
        for i in range(3):
            tP[i]+=P[i]
    for i in range(3):
        tP[i]/=N
    opt_x=-tP[1]/(2*tP[2])
    scrs=[val(tP,locsversions[ITER][i]) for i in range(N)]
    print('{},{},{},{}'.format(seed,val(tP,opt_x),np.mean(scrs),np.median(scrs)))
    fout.write('{},{},{},{}\n'.format(seed,val(tP,opt_x),np.mean(scrs),np.median(scrs)))
    print('time='+str(time.time()-st))
fout.close()

'''t1=np.linspace(min(locs[0:N])-20,max(locs[0:N])+20,50)
fig,ax=plt.subplots(1,3,figsize=(20,5))

ax[0].set_title('F_i, MEAN(F_i), agents\' final parameters')
ax[0].plot(t1,val(tP,t1),linewidth=3,alpha=0.75)
for P in Ps:
    ax[0].plot(t1,val(P,t1),linewidth=1,alpha=0.15)
ax[0].scatter(x=locs[0:N],y=[val(tP,locs[i]) for i in range(N)],c='black')

ax[1].set_title('agents\' parameters')
ax[1].set_xlabel('agent\'s position in graph')
for k in range(0,ITER):
    ax[1].plot(locsversions[k],linewidth=1,color=(k/ITER,0,1-k/ITER,0.2))
ax[1].plot([0,N],[opt_x,opt_x])

ax[2].set_title('agents\' scores')
ax[2].set_xlabel('agent\'s position in graph')
for k in range(0,ITER):
    ax[2].plot([val(tP,locsversions[k][i]) for i in range(N)],linewidth=1,color=(k/ITER,0,1-k/ITER,0.2))
ax[2].plot([0,N],[val(tP,opt_x),val(tP,opt_x)])

#plt.savefig('N,A,ITER,breadth={},{},{},{}'.format(N,A,ITER,breadth)+'.png')'''

1,1662.8954346209405,1665.9472666387735,1665.7500616583006
time=0.8055417537689209
2,2042.474950471248,2092.0393945359574,2096.882976198526
time=1.6065268516540527
3,1478.4005600825353,1492.766511004088,1491.4576587585334
time=2.8348305225372314
4,1514.0228227724397,1514.9061681385342,1514.5303138258375
time=3.5434250831604004
5,2004.9142775692517,2007.4600026786766,2006.6181499574905
time=4.258539199829102
6,1878.7940106579588,1879.8501256217905,1879.779260203012
time=4.9761552810668945
7,1361.824926502943,1362.4026684531077,1361.9715855706338
time=5.662773370742798
8,1306.686798255798,1311.9430880225098,1311.0162338280493
time=6.376354932785034
9,2090.260944820599,2114.163990547509,2113.5987975376966
time=7.085960865020752
10,1635.6923804040312,1675.6628737434758,1675.3494937296382
time=7.879498243331909
11,1592.7901154069762,1597.5703480831344,1597.5326995777439
time=8.595090627670288
12,1238.525196998235,1255.0629336533755,1253.7100710350787
time=9.302688360214233
13,1424.116878232

In [ ]:
tmp={}
DFs={}
FORM='ret/simple_mean(N,A,breadth,ITER,T={},{},{},{},{}).txt'
Fs=[]
params=[[50,0,b,800,0.01] for b in range(1,5)]
for p in params:
    Fs.append(FORM.format(*p))
plt.figure()
for F in Fs:
    fin=open(F,'r')
    txt=[l for l in fin]
    #print(txt)
    names=txt[0][0:-1].split(',')
    infos={}
    for n in names:
        infos[n]=[]
    for i in range(1,len(txt)):
        info=list(map(lambda x:float(x),txt[i][0:-1].split(',')))
        info[0]=int(info[0])
        for j in range(len(info)):
            infos[names[j]].append(info[j])
    DFs[F]=pd.DataFrame(infos)
    print(DFs[F])
    print(np.array(DFs[F]['mean']-DFs[F]['opt']))
    plt.hist(np.array(DFs[F]['mean']-DFs[F]['opt']))